## Utils

In [ ]:
!pip install --upgrade langchain_community langchain_core langchain_openai langgraph pypdf langchain_groq fastembed chromadb

In [ ]:
# Utils
def generate_alphanumeric_code(length=18):
    characters = string.ascii_uppercase + string.digits
    code = "".join(random.choices(characters, k=length))
    return code

from langchain_core.messages import ToolMessage
from langchain_core.runnables import RunnableLambda

from langgraph.prebuilt import ToolNode


def handle_tool_error(state) -> dict:
    error = state.get("error")
    tool_calls = state["messages"][-1].tool_calls
    return {
        "messages": [
            ToolMessage(
                content=f"Error: {repr(error)}\n please fix your mistakes.",
                tool_call_id=tc["id"],
            )
            for tc in tool_calls
        ]
    }


def create_tool_node_with_fallback(tools: list) -> dict:
    return ToolNode(tools).with_fallbacks(
        [RunnableLambda(handle_tool_error)], exception_key="error"
    )


def _print_event(event: dict, _printed: set, max_length=1500):
    current_state = event.get("dialog_state")
    if current_state:
        print("Currently in: ", current_state[-1])
    message = event.get("messages")
    if message:
        if isinstance(message, list):
            message = message[-1]
        if message.id not in _printed:
            msg_repr = message.pretty_repr(html=True)
            if len(msg_repr) > max_length:
                msg_repr = msg_repr[:max_length] + " ... (truncated)"
            print(msg_repr)
            _printed.add(message.id)

# Basics

## LangChain and LangGraph

### [LangChain Introduction](https://python.langchain.com/docs/introduction/)

LangChain implements a standard interface for large language models and related technologies, such as embedding models and vector stores, and integrates with hundreds of providers. 


```python
    from langchain.chat_models import init_chat_model
    model = init_chat_model("llama3-8b-8192", model_provider="groq")
    model.invoke("Hello, world!")
```


### [LangGraph Introduction](https://langchain-ai.github.io/langgraph/)

Orchestration framework for combining LangChain components into production-ready applications with persistence, streaming, and other key features.

```python
    from typing import Annotated
    from typing_extensions import TypedDict
    from langgraph.graph import StateGraph, START, END
    from langgraph.graph.message import add_messages
    from langchain_anthropic import ChatAnthropic
    llm = ChatAnthropic(model="claude-3-5-sonnet-20240620")
    class State(TypedDict):
        messages: Annotated[list, add_messages]
    def chatbot(state: State):
        return {"messages": [llm.invoke(state["messages"])]}
    graph_builder = StateGraph(State)
    graph_builder.add_node("chatbot", chatbot)
    graph_builder.add_edge(START, "chatbot")
    graph_builder.add_edge("chatbot", END)
```

![Basic LangGraph](../imgs/langgraph_basic.png)

In [ ]:
import os
groq_key = "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"
os.environ["GROQ_API_KEY"] = groq_key

## AI Agents

![AI Agents](../imgs/AI_Agents.jpg)



In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import AnyMessage, add_messages

class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import Runnable, RunnableConfig


class Assistant:
    def __init__(self, runnable: Runnable):
        self.runnable = runnable

    def __call__(self, state: State, config: RunnableConfig):
        while True:
            configuration = config.get("configurable", {})
            result = self.runnable.invoke(state)
            # If the LLM happens to return an empty response, we will re-prompt it
            # for an actual response.
            if not result.tool_calls and (
                not result.content
                or isinstance(result.content, list)
                and not result.content[0].get("text")
            ):
                messages = state["messages"] + [("user", "Respond with a real output.")]
                state = {**state, "messages": messages}
            else:
                break
        return {"messages": result}

In [ ]:
@tool
def get_traffic_conditions(config: RunnableConfig, city):
    traffic_conditions = {
        'berlin': 'normal',
        'munich': 'very busy',
        'frankfurt': 'very busy'
    }
    return traffic_conditions[city.lower()]

@tool
def get_weather_conditions(config: RunnableConfig, city):
    weather_conditions = {
        'berlin': '5C, rainy',
        'munich': '2C, windy',
        'frankfurt': '-1C, snowy'
    }
    return weather_conditions[city.lower()]


In [ ]:
llm = ChatGroq(model="llama-3.3-70b-versatile")

primary_assistant_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant who provides traffic and weather information"
            " Use the provided tools to get the traffic and weather conditions for a city.",
        ),
        ("placeholder", "{messages}"),
    ]
)

tools = [
    get_traffic_conditions,
    get_weather_conditions
]

assistant_runnable = primary_assistant_prompt | llm.bind_tools(tools)

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import tools_condition

builder = StateGraph(State)


# Define nodes: these do the work
builder.add_node("assistant", Assistant(assistant_runnable))
builder.add_node("tools", create_tool_node_with_fallback(tools))
# Define edges: these determine how the control flow moves
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    tools_condition,
)
builder.add_edge("tools", "assistant")

# The checkpointer lets the graph persist its state
# this is a complete memory for the entire graph.
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph(xray=True).draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [ ]:
import uuid

thread_id = str(uuid.uuid4())
config = {
    "configurable": {
        # The passenger_id is used in our flight tools to
        # fetch the user's flight information
        "car_number": "DYY2750",
        # Checkpoints are accessed by thread_id
        "thread_id": thread_id,
    }
}

questions = [
    "Can you tell me the traffic conditions in Berlin?",
    "I want to travel from munich to frankfurt, do I need an unmbrella?",
]

In [ ]:
_printed = set()
for question in questions:
    events = graph.stream(
        {"messages": ("user", question)}, config, stream_mode="values"
    )
    for event in events:
        _print_event(event, _printed)

## Retrival Augmented Generation (RAG)

![Retrival Augmented Generation](../imgs/RAG.jpg)

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
import os
from glob import glob
from langchain_groq import ChatGroq

In [ ]:
llm = ChatGroq(model="llama-3.3-70b-versatile")

embeddings = FastEmbedEmbeddings(model_name="BAAI/bge-base-en-v1.5")

vector_store = InMemoryVectorStore(embeddings)

In [ ]:
def load_documents(document_path):
    loader = PyPDFLoader(document_path)
    pages = []
    for page in loader.lazy_load():
      pages.append(page)
    _ = vector_store.add_documents(pages)
    print(f"Loaded {len(pages)} pages from {document_path}")
    return 0

In [ ]:
@tool(response_format="content_and_artifact")
def retrieve(query: str):
    """Retrieve information related to a query."""
    retrieved_docs = vector_store.similarity_search(query, k=1)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\n" f"Content: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [ ]:
# Step 1: Generate an AIMessage that may include a tool-call to be sent.
def query_or_respond(state: MessagesState):
    """Generate tool call for retrieval or respond."""
    llm_with_tools = llm.bind_tools([retrieve])
    response = llm_with_tools.invoke(state["messages"])
    # MessagesState appends messages to state instead of overwriting
    return {"messages": [response]}

# Step 2: Execute the retrieval.
tools = ToolNode([retrieve])

# Step 3: Generate a response using the retrieved content.
def generate(state: MessagesState):
    """Generate answer."""
    # Get generated ToolMessages
    recent_tool_messages = []
    for message in reversed(state["messages"]):
        if message.type == "tool":
            recent_tool_messages.append(message)
        else:
            break
    tool_messages = recent_tool_messages[::-1]

    # Format into prompt
    docs_content = "\n\n".join(doc.content for doc in tool_messages)
    system_message_content = (
        "You are an assistant for question-answering tasks. "
        "Use the following pieces of retrieved context to answer the question. "
        "Use three sentences maximum and keep the answer concise."
        "Include the reference that is being used from the retrieved context."
        "If you do not get any retrived content below, say that you do not know about it."
        "Do not give any additional information except for what is in the documents."
        "\n\n # Retrived Documents:"
        f"{docs_content}"
    )
    conversation_messages = [
        message
        for message in state["messages"]
        if message.type in ("human", "system")
        or (message.type == "ai" and not message.tool_calls)
    ]
    prompt = [SystemMessage(system_message_content)] + conversation_messages

    # Run
    response = llm.invoke(prompt)
    return {"messages": [response]}

In [ ]:
graph_builder.add_node(query_or_respond)
graph_builder.add_node(tools)
graph_builder.add_node(generate)

graph_builder.set_entry_point("query_or_respond")
graph_builder.add_conditional_edges(
    "query_or_respond",
    tools_condition,
    {END: END, "tools": "tools"},
)
graph_builder.add_edge("tools", "generate")
graph_builder.add_edge("generate", END)

graph = graph_builder.compile()